In [2]:
from transformers import pipeline

## 1. Preprocessing with a tokenizer

In [6]:
# load the tokenizer
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [8]:
raw_inputs = [
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much!",
]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")
print(inputs) #return tensor for tokens and attention_mask

{'input_ids': tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102],
        [  101,  1045,  5223,  2023,  2061,  2172,   999,   102,     0,     0,
             0,     0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]])}


## 2. Going through the model

In [10]:
from transformers import AutoModel

# Load a pretrained model
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModel.from_pretrained(checkpoint)

In [13]:
# create the hiddle state after embedding and attention layers; the hidden state is to capture the "meaning" of each token in its context
outputs = model(**inputs)
print(outputs.last_hidden_state.shape)
# 2 refers to how many sentences, 16 refers to how many tokens in each sentncees, 768 refers to the number of dimensions in each token’s "magic" embedding vector

torch.Size([2, 16, 768])


## 3. Model heads: Making sense out of numbers

In [15]:
# adding model head
# For sentiment analysis, use a model with a sequence classification head
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
outputs = model(**inputs)
print(outputs.logits)
#Sentence 1: Scores for [NEGATIVE, POSITIVE] 
#Sentence 2: Scores for [NEGATIVE, POSITIVE]

tensor([[-1.5607,  1.6123],
        [ 4.1692, -3.3464]], grad_fn=<AddmmBackward0>)


## 4. Postprocessing the output

In [16]:
print(outputs.logits)

tensor([[-1.5607,  1.6123],
        [ 4.1692, -3.3464]], grad_fn=<AddmmBackward0>)


In [20]:
import torch

predictions = torch.nn.functional.softmax(outputs.logits, dim=-1) #use softmax function to turn logits into probablities 
print(predictions)

tensor([[4.0195e-02, 9.5981e-01],
        [9.9946e-01, 5.4418e-04]], grad_fn=<SoftmaxBackward0>)


In [21]:
model.config.id2label

{0: 'NEGATIVE', 1: 'POSITIVE'}